In [7]:
import numpy as np
import time

###### 数组在内存中的排列方式

In [8]:
row_major = np.zeros((10, 10), order = 'C') # C-type

In [9]:
col_major = np.zeros((10, 10), order = 'F') # F-type

###### actions on axis: 按需选择 C 或者 F type 的数组

In [13]:
a = np.zeros((500, 500), order = 'C')
b = np.zeros((500, 500), order = 'F')
N = 10000

# np.concatenate 按指定的axis拼接数组
def f1(a):
    for _ in range(N):
        np.concatenate((a, a), axis = 0) 
def f2(b):
    for _ in range(N):
        np.concatenate((b, b), axis = 0)

t0 = time.time()
f1(a)
t1 = time.time()
f2(b)
t2 = time.time()

print((t1 - t0) / N)
print((t2 - t1) / N)

0.0003237801551818848
0.0003559266090393066


###### np._stack vs np.concatenate: 后者略快

In [21]:
%time np.vstack((a, a))

CPU times: user 2.54 ms, sys: 2.41 ms, total: 4.95 ms
Wall time: 4.4 ms


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
%time np.concatenate((a, a), axis = 0)

CPU times: user 4.32 ms, sys: 764 µs, total: 5.08 ms
Wall time: 3.91 ms


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
%time np.hstack((b, b))

CPU times: user 206 µs, sys: 4.89 ms, total: 5.1 ms
Wall time: 3.95 ms


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
%time np.concatenate((b, b), axis = 1)

CPU times: user 3.37 ms, sys: 1.25 ms, total: 4.61 ms
Wall time: 3.52 ms


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

###### Copy慢, View快

In [25]:
a = np.arange(1, 7).reshape((3, 2))

In [26]:
a_view = a[:2] #切片操作是view，得到的对象和原对象指向相同的内存

In [28]:
a_copy = a[:2].copy() #copy了元对象，二者的内存不同
a_copy[1, 1] = 0

In [29]:
print(a)
print(a_copy)

[[1 2]
 [3 4]
 [5 6]]
[[1 2]
 [3 0]]


In [30]:
a_view[1, 1] = 0
print(a)

[[1 2]
 [3 0]
 [5 6]]


In [34]:
a = np.zeros((1000, 1000))
b = np.zeros((1000, 1000))
N = 10000

def f1(a):
    for _ in range(N):
        a *= 2 # same as a[:] *= 2, view后改变其值
def f2(b):
    for _ in range(N):
        b = 2 * b #复制了b之后再赋值！慢！

t0 = time.time()
f1(a)
t1 = time.time()
f2(b)
t2 = time.time()

print('%f' % ((t1 - t0) / N))
print('%f' % ((t2 - t1) / N))

0.001010
0.001636


###### np.flatten vs np.ravel

In [35]:
def f1(a):
    for _ in range(N):
        a.flatten() #flatten总是返回一个copy
def f2(b):
    for _ in range(N):
        b.ravel() #ravel 只会在需要复制的时候复制

t0 = time.time()
f1(a)
t1 = time.time()
f2(b)
t2 = time.time()

print('%f' % ((t1 - t0) / N))
print('%f' % ((t2 - t1) / N))

0.001034
0.000001


###### 选择数据能用view就用view

In [37]:
a_view1 = a[1 : 2, 3 : 6] # 切片 slice
a_view2 = a[: 100]        # 同上
a_view3 = a[::2]          # 跳步切片
a_view4 = a.ravel()

In [40]:
a_copy1 = a[[1, 4, 6], [2, 4, 6]]        #用index选择而非切片
a_copy2 = a[[True, True], [False, True]] #用mask
a_copy3 = a[[1, 2], :]                   #虽然1， 2 连在一起了但是这么写也是copy
a_copy4 = a[a[1, :] != 0, :]             #fancy indexing
a_copy5 = a[np.isnan(a), :]              #fancy indexing

###### 加速fancy indexing

In [46]:
a = np.random.rand(1000000, 10)
N = 100
indices = np.random.randint(0, 1000000, size= 10000)

def f1(a):
    for _ in range(N):
        _ = np.take(a, indices, axis = 0) # 使用 np.take 速度更快
def f2(b):
    for _ in range(N):
        _ = b[indices, :]


t0 = time.time()
f1(a)
t1 = time.time()
f2(a)
t2 = time.time()

print('____%f' % ((t1 - t0) / N))
print('____%f' % ((t2 - t1) / N))

____0.000242
____0.000463


In [47]:
mask = a[:, 0] < .5

def f1(a):
    for _ in range(N):
        _ = np.compress(mask, a, axis = 0) # np.compress更快
def f2(b):
    for _ in range(N):
        _ = b[mask]

t0 = time.time()
f1(a)
t1 = time.time()
f2(a)
t2 = time.time()

print('____%f' % ((t1 - t0) / N))
print('____%f' % ((t2 - t1) / N))

____0.038728
____0.050000


###### out参数: https://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs

In [48]:
%time a = a + 1

CPU times: user 21 ms, sys: 37.1 ms, total: 58.1 ms
Wall time: 56.4 ms


In [49]:
%time a += 1

CPU times: user 13.4 ms, sys: 1.6 ms, total: 15 ms
Wall time: 13.6 ms


In [50]:
%time np.add(a, 1, out = a) #使用 out参数避免拷贝

CPU times: user 12.9 ms, sys: 1 ms, total: 13.9 ms
Wall time: 12.1 ms


array([[3.54069066, 3.9701559 , 3.02997936, ..., 3.24251098, 3.01782725,
        3.82279987],
       [3.12372207, 3.19550119, 3.64057795, ..., 3.12862376, 3.01146194,
        3.89737635],
       [3.11066837, 3.19489798, 3.27089342, ..., 3.16116508, 3.56775441,
        3.70456071],
       ...,
       [3.10498744, 3.65362937, 3.68817647, ..., 3.38307019, 3.71989335,
        3.73359843],
       [3.75121717, 3.28836679, 3.80950858, ..., 3.83526388, 3.83440426,
        3.3321167 ],
       [3.44626113, 3.56143754, 3.54740745, ..., 3.13915841, 3.56637279,
        3.76211531]])

###### 如pandas那样给列起名字

In [51]:
import pandas as pd

In [53]:
# numpy 要比 pandas 快得多
a = np.zeros(3, dtype = [('foo', np.int32), ('bar', np.float16)]) 
b = pd.DataFrame(np.zeros((3, 2), dtype = np.int32), columns = ['foo', 'bar'])
b['bar'] = b['bar'].astype(np.float16)

In [57]:
def f1(a):
    for _ in range(N):
        a['bar'] *= a['foo']
def f2(b):
    for _ in range(N):
        b['bar'] *= b['foo']

t0 = time.time()
f1(a)
t1 = time.time()
f2(b)
t2 = time.time()

print('____%f' % ((t1 - t0) / N))
print('____%f' % ((t2 - t1) / N))

____0.000046
____0.000823
